In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner as kt
import numpy as np

In [3]:
# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

# Normalize data (scale pixel values between 0 and 1)
x_train, x_test = x_train / 255.0, x_test / 255.0

# Convert labels to categorical
# Convert labels to categorical format (one-hot encoding)
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)


In [4]:
# Define a basic ANN model
def build_ann_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape=(32, 32, 3)))  # Flatten image
    
    # Tune the number of neurons in hidden layers
    for i in range(hp.Int("num_layers", 1, 3)):  # Choose 1-3 layers
        model.add(layers.Dense(
            units=hp.Int(f"units_{i}", min_value=64, max_value=512, step=64),
            activation=hp.Choice("activation", ["relu", "tanh"])
        ))

    model.add(layers.Dense(10, activation="softmax"))  # Output layer

    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice("learning_rate", [1e-2, 1e-3, 1e-4])
        ),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    
    return model

In [5]:
ann_model = build_ann_model(kt.HyperParameters())
ann_model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

C:\ProgramData\anaconda3\envs\tf_env\lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 25s 14ms/step - accuracy: 0.0995 - loss: 2.6656 - val_accuracy: 0.1000 - val_loss: 2.3043
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 22s 14ms/step - accuracy: 0.1026 - loss: 2.3037 - val_accuracy: 0.1000 - val_loss: 2.3048
Epoch 3/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 19s 12ms/step - accuracy: 0.1020 - loss: 2.3041 - val_accuracy: 0.1000 - val_loss: 2.3053
Epoch 4/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 22s 14ms/step - accuracy: 0.0992 - loss: 2.3040 - val_accuracy: 0.1000 - val_loss: 2.3031
Epoch 5/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 20s 13ms/step - accuracy: 0.0984 - loss: 2.3041 - val_accuracy: 0.1000 - val_loss: 2.3035
Epoch 6/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 20s 13ms/step - accuracy: 0.0984 - loss: 2.3039 - val_accuracy: 0.1000 - val_loss: 2.3039
Epoch 7/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 19s 12ms/step - accuracy: 0.1003 - loss: 2.3039 - val_accuracy: 0.1000 - val_loss: 2.3042
Epoch 8/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 22s 14ms/step - accuracy: 0.0995 -

In [6]:
# Predict
predictions = np.argmax(ann_model.predict(x_test), axis=1)
print(predictions[:10])  # Show first 10 predictions

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step
[9 9 9 9 9 9 9 9 9 9]


In [7]:
tuner = kt.RandomSearch(
    build_ann_model,
    objective="val_accuracy",
    max_trials=5,
    executions_per_trial=1,
    directory="keras_tuner",
    project_name="cifar10_ann"
)

In [8]:
tuner.search(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

Trial 5 Complete [00h 06m 01s]
val_accuracy: 0.5127000212669373

Best val_accuracy So Far: 0.5127000212669373
Total elapsed time: 00h 27m 50s
